# Simulating EPA-RE using points of low-order

As visible in the [`formulas`](formulas.ipynb) notebook, most addition formulas have exceptional cases.
We can use trigger these exceptions by supplying points of low order to the scalar multiplier, which
can result in the point at infinity appearing in an unexpected place in the scalar multiplication computation.
The behavior of an implementation with regards to these exceptional cases depends on what formulas it implements
and what checks it performs on the inputs/outputs/intermediates of the formulas and the whole scalar multiplication.
Furthermore, what multiples appear in the scalar multiplication depends on the scalar multiplier.

This notebook explores the statistical differences in error rates of scalar multipliers computing on low-order base points
to reverse-engineer the scalar multipliers and countermeasures, even in the presence of scalar randomization. This
is possible because we do not care about the actual multiples that happen for any given scalar, but rather the statistical
properties, i.e.: For random scalars, how probable is an error for a point of low order, for example, 5?

Examine the figure below that shows the big picture.
![](unravelling.svg)

In [3]:
import itertools
import gc
import glob
import hashlib
import pickle
import random
import re
import tempfile

import warnings
warnings.filterwarnings(
    "ignore",
    message="pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html.",
    category=UserWarning
)

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter
from pathlib import Path
from random import randint, randbytes
from typing import Type, Any

from tqdm.auto import tqdm, trange

from pyecsca.ec.params import DomainParameters, get_params
from pyecsca.ec.mult import *
from pyecsca.ec.mod import mod
from pyecsca.sca.re.rpa import multiple_graph
from pyecsca.sca.re.epa import graph_to_check_inputs, evaluate_checks
from pyecsca.misc.utils import TaskExecutor

from common import *

## Initialize
Let's first initialize some sets of multipliers and countermeasures we will be reverse-engineering. They come from the
[common.py](common.py) file, which you can examine for more information. We need to silence [some warnings](https://github.com/newaetech/chipwhisperer/pull/549), due to 
ChipWhisperer, which is used by pyecsca.

In [4]:
def silence():
    import warnings
    warnings.filterwarnings(
        "ignore",
        message="pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html.",
        category=UserWarning
    )
silence()

In [5]:
nmults = len(all_mults)
nmults_ctr = len(all_mults_with_ctr)
nerror_models = len(all_error_models)
ncfgs = nmults_ctr * nerror_models

print(f"Scalar multipliers considered:  {nmults}")
print(f"Scalar multipliers (with a single countermeasure) considered:  {nmults_ctr}")
print(f"Error models considered:  {nerror_models}")
print(f"Total configurations considered:  {ncfgs}")

Scalar multipliers considered:  65
Scalar multipliers (with a single countermeasure) considered:  390
Error models considered:  32
Total configurations considered:  12480


## Prepare
Next, let's setup some parameters. The curve specified below is not used for any computation, except that its order is given
to the multipliers. The `use_init` and `use_multiply` booleans specify whether the precomputation part and multiplication part, respectively, is used for the simulation. The `num_workers` option specifies the level of parallelization (via processes) that is employed. Make sure to set this to something reasonable. The `samples` option specifies the amount of samples {i.e. scalar multiplication per scalar multiplier) that get computed in one *chunk*, which corresponds to one run
of the cell. We use chunks to make it easy to compute and aggregate more samples, without needing too much memory.

In [ ]:
category = "secg"
curve = "secp256r1"
params = get_params(category, curve, "projective")
bits = params.order.bit_length()

use_init = True
use_multiply = True

num_workers = 30
samples = 1000

selected_mults = all_mults

In [ ]:
def simulate_multiples(mult: MultIdent,
                       params: DomainParameters,
                       bits: int,
                       samples: int = 100,
                       use_init: bool = True,
                       use_multiply: bool = True,
                       seed: bytes | None = None) -> MultResults:
    """
    Takes a MultIdent, which specifies a scalar multiplier (with an optional countermeasure)
    and simulates `samples` scalar multiplications, while tracking which multiples of the
    symbolic input point get computed.
    """
    results = []
    if seed is not None:
        random.seed(seed)

    # If no countermeasure is used, we have fully random scalars.
    # Otherwise, fix one per chunk.
    if mult.countermeasure is None:
        scalars = [random.randint(1, 2**bits) for _ in range(samples)]
    else:
        one = random.randint(1, 2**bits)
        scalars = [one for _ in range(samples)]

    for scalar in scalars:
        results.append(multiple_graph(scalar, params, mult.klass, mult.partial, use_init, use_multiply))
    return MultResults(results, samples)

In [ ]:
def simulate_multiples_direct(mult: MultIdent,
                              params: DomainParameters,
                              bits: int,
                              fname: str,
                              samples: int = 100,
                              use_init: bool = True,
                              use_multiply: bool = True,
                              seed: bytes | None = None) -> str:
    """
    Like the `simulate_multiples` function above, but stores the pickled output directly
    into a file named `fname`.
    """
    results = []
    if seed is not None:
        random.seed(seed)

    # If no countermeasure is used, we have fully random scalars.
    # Otherwise, fix one per chunk.
    if mult.countermeasure is None:
        scalars = [random.randint(1, 2**bits) for _ in range(samples)]
    else:
        one = random.randint(1, 2**bits)
        scalars = [one for _ in range(samples)]

    for scalar in scalars:
        results.append(multiple_graph(scalar, params, mult.klass, mult.partial, use_init, use_multiply))
    result = MultResults(results, samples)
    with open(fname, "wb") as f:
        pickle.dump((mult, result), f)
    return fname

In [ ]:
def evaluate_multiples(mult: MultIdent, res: MultResults, divisors: set[int]):
    """
    Takes MultIdent and MultResults and a set of divisors (base point orders `q`) and
    evaluates them using the error model from the MultIdent. Note that the MultIdent
    must have an error model in this case. Returns the ProbMap.
    """
    errors = {divisor: 0 for divisor in divisors}
    samples = len(res)
    divisors_hash = hashlib.blake2b(str(sorted(divisors)).encode(), digest_size=8).digest()
    for ctx, out in res:
        check_inputs = graph_to_check_inputs(ctx, out,
                                             check_condition=mult.error_model.check_condition,
                                             precomp_to_affine=mult.error_model.precomp_to_affine)
        for q in divisors:
            error = evaluate_checks(check_funcs={"add": mult.error_model.check_add(q), "affine": mult.error_model.check_affine(q)},
                                    check_inputs=check_inputs)
            errors[q] += error
    # Make probmaps smaller. Do not store zero probabilities.
    probs = {}
    for q, error in errors.items():
        if error != 0:
            probs[q] = error / samples
    return ProbMap(probs, divisors_hash, samples)

In [ ]:
def evaluate_multiples_direct(mult: MultIdent, fname: str, offset: int, divisors: set[int]):
    """
    Like `evaluate_multiples`, but instead reads the MultResults from a file named `fname`
    at an `offset`. Still returns the ProbMap, which is significantly smaller and easier
    to pickle than the MultResults.
    """
    with open(fname, "rb") as f:
        f.seek(offset)
        _, res = pickle.load(f)
    errors = {divisor: 0 for divisor in divisors}
    samples = len(res)
    divisors_hash = hashlib.blake2b(str(sorted(divisors)).encode(), digest_size=8).digest()
    for ctx, out in res:
        check_inputs = graph_to_check_inputs(ctx, out,
                                             check_condition=mult.error_model.check_condition,
                                             precomp_to_affine=mult.error_model.precomp_to_affine)
        for q in divisors:
            error = evaluate_checks(check_funcs={"add": mult.error_model.check_add(q), "affine": mult.error_model.check_affine(q)},
                                    check_inputs=check_inputs)
            errors[q] += error
    # Make probmaps smaller. Do not store zero probabilities.
    probs = {}
    for q, error in errors.items():
        if error != 0:
            probs[q] = error / samples
    return ProbMap(probs, divisors_hash, samples)

## Run
Run this cell as many times as you want. It will simulate `samples` scalar multiplications for each `MultIdent` (a scalar multiplier implementation with an optional countermeasure) and store them into the chunk.

In [ ]:
chunk_id = randbytes(4).hex()
with TaskExecutor(max_workers=num_workers, initializer=silence) as pool, tempfile.TemporaryDirectory() as tmp_dirname:
    tmp_path = Path(tmp_dirname)
    for i, mult in enumerate(all_mults_with_ctr):
        pool.submit_task(mult,
                         simulate_multiples_direct,
                         mult, params, bits, tmp_path / f"{i}.pickle", samples, seed=chunk_id, use_init=use_init, use_multiply=use_multiply)
    with open(f"multiples_{bits}_{'init' if use_init else 'noinit'}_{'mult' if use_multiply else 'nomult'}_chunk{chunk_id}.pickle","wb") as h:
        for mult, future in tqdm(pool.as_completed(), desc="Computing multiple graphs.", total=len(pool.tasks)):
            print(f"Got {mult}.")
            if error := future.exception():
                print("Error!", error)
                continue
            fpath = future.result()
            with fpath.open("rb") as f:
                h.write(f.read())
            fpath.unlink()

## Process
The multiple chunks generated in the previous cell are not fully processed yet. They only contain a trace of the scalar multiplication computation as a graph of formula applications on symbolic multiples of the input point. In the next step,
we iterate over all possible error models and evaluate these graphs using the error models. Thereby, we go from e.g. 1000
scalar multiplication traces + error model + base point order `q` to a number from 0 to 1000 specifying the number of errors in those  multiplications given the error model and base point order `q`. The orders that are used are described in the
[common.py](common.py) file.

> The cell below computes the error probabilities for all chunks of multiples that are missing them. Hence you only need
> to run it once.

In [ ]:
with TaskExecutor(max_workers=num_workers, initializer=silence) as pool:
    for in_fname in tqdm(glob.glob(f"multiples_{bits}_{'init' if use_init else 'noinit'}_{'mult' if use_multiply else 'nomult'}_chunk*.pickle"), desc="Processing chunks", smoothing=0):
        
        match = re.match("multiples_(?P<bits>[0-9]+)_(?P<init>(?:no)?init)_(?P<mult>(?:no)?mult)_chunk(?P<id>[0-9a-f]+).pickle", in_fname)
        chunk_id = match.group("id")
        out_fname = f"probs_{bits}_{use_init}_{use_multiply}_chunk{chunk_id}.pickle"

        in_file = Path(in_fname)
        out_file = Path(out_fname)

        cfgs_todo = set()
        for mult in all_mults_with_ctr:
            for error_model in all_error_models:
                cfgs_todo.add(mult.with_error_model(error_model))

        if out_file.exists():
            print(f"Processing chunk {chunk_id}, some(or all) probmaps found.")
            with out_file.open("r+b") as f:
                while True:
                    try:
                        full, _ = pickle.load(f)
                        cfgs_todo.remove(full)
                        last_end = f.tell()
                    except EOFError:
                        break
                    except pickle.UnpicklingError:
                        f.truncate(last_end)
            if not cfgs_todo:
                print(f"Chunk complete. Continuing...")
                continue
            else:
                print(f"Chunk missing {len(cfgs_todo)} probmaps, computing...")
        else:
            print(f"Processing chunk {chunk_id}, no probmaps found.")
        
        with in_file.open("rb") as f, out_file.open("ab") as h:
            loading_bar = tqdm(total=nmults_ctr, desc=f"Loading chunk {chunk_id}.", smoothing=0)
            processing_bar = tqdm(total=len(cfgs_todo), desc=f"Processing {chunk_id}.", smoothing=0)
            while True:
                try:
                    start = f.tell()
                    mult, vals = pickle.load(f)
                    loading_bar.update(1)
                    for error_model in all_error_models:
                        full = mult.with_error_model(error_model)
                        if full in cfgs_todo:
                            # Pass the file name and offset to speed up computation start.
                            pool.submit_task(full,
                                             evaluate_multiples_direct,
                                             full, in_fname, start, divisor_map["all"])
                    gc.collect()
                    if len(pool.tasks) > 1000:
                        for full, future in pool.as_completed():
                            processing_bar.update(1)
                            if error := future.exception():
                                print("Error!", full, error)
                                continue
                            res = future.result()
                            pickle.dump((full, res), h)
                except EOFError:
                    break
                except pickle.UnpicklingError:
                    print("Bad unpickling, the multiples file is likely truncated.")
                    break
            for full, future in pool.as_completed():
                processing_bar.update(1)
                if error := future.exception():
                    print("Error!", full, error)
                    continue
                res = future.result()
                pickle.dump((full, res), h)
        print("Chunk completed.")


## Merge

Finally, we have multiple chunks of error probabilities (i.e. `ProbMaps`) that we merge into a single file `merged.pickle`
that can be used by the [visualize](visualize.ipynb) and [distinguish](distinguish.ipynb) notebooks.

In [ ]:
probmaps = {}
for in_fname in tqdm(glob.glob(f"probs_{bits}_{'init' if use_init else 'noinit'}_{'mult' if use_multiply else 'nomult'}_chunk*.pickle"), desc="Processing chunks", smoothing=0):
    
    match = re.match("probs_(?P<bits>[0-9]+)_(?P<init>(?:no)?init)_(?P<mult>(?:no)?mult)_chunk(?P<id>[0-9a-f]+).pickle", in_fname)
    chunk_id = match.group("id")    
    
    with open(in_fname, "rb") as f:
        loading_bar = tqdm(total=ncfgs, desc=f"Loading chunk {chunk_id}.", smoothing=0)
        while True:
            try:
                mult, probmap = pickle.load(f)
                if mult in probmaps:
                    current = probmaps[mult]
                    current.merge(probmap)
                else:
                    probmaps[mult] = probmap
                loading_bar.update(1)
            except EOFError:
                break
            except pickle.UnpicklingError:
                print("Bad unpickling, the probs file is likely truncated.")
                break
        loading_bar.close()

            

In [ ]:
with open("merged.pickle", "wb") as f:
    pickle.dump(probmaps, f)

## Misc

The cell below performs some profiling of the multiple evaluation.

In [ ]:
from pyinstrument import Profiler as PyProfiler
mult = next(iter(multiples_mults))
res = multiples_mults[mult]

for checks in powerset(checks_add):
    for precomp_to_affine in (True, False):
        for check_condition in ("all", "necessary"):
            error_model = ErrorModel(checks, check_condition=check_condition, precomp_to_affine=precomp_to_affine)
            full = mult.with_error_model(error_model)
            print(full)
            with PyProfiler() as prof:
                probmap = evaluate_multiples(full, res, divisor_map["all"])
            print(prof.output_text(unicode=True, color=True))
            break
        break
    break